In [2]:
import torch
import torch.nn as nn
import numpy as np
import math

In [11]:
class Attention(nn.Module):
    '''
    Experimental implementation of the scaled dot-product attention mechanism
    and not yet fully implemented
    '''
    def __init__(self, embed_size, heads):
        super(Attention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embed size needs to be divisible by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query, mask=None):
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split the embedding into self
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        # queries = query.reshape(N, key_len, self.heads, self.head_dim)
        queries = query.reshape(N, query_len, self.heads, self.head_dim)

        values = self.values(values)
        keys = self.keys(keys)
        queries = self.queries(queries)

        # Permute to (N, heads, seq_len, head_dim) for attention
        queries = queries.permute(0, 2, 1, 3)
        keys = keys.permute(0, 2, 1, 3)
        values = values.permute(0, 2, 1, 3)

        energy  = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        # queries shape: (N, query_len, heads, head_dim)
        # keys shape: (N, key_len, heads, head_dim)
        # energy shape: (N, heads, query_len, key_len)

        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))
        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)

        out = torch.einsum("nqhd,nkhd->nhqk", [query, keys])
        # attention shape: (N, heads, query_len, key_len)
        # values shape: (N, value_len, heads, heads_dim)
        # out shape: (N, query_len, heads, head_dim)
        
        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )
        # attention shape: (N, heads, query_len, key_len)
        # values shape: (N, value_len, heads, heads_dim) 
        # (N, query_len, heads, head_dim) flatten after einsum

        out = self.fc_out(out)

        # For demonstration purposes, we'll just return the reshaped tensors
        return out

class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = Attention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)

        # Add skip connection, run through normalization and finally dropout
        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out

class Encoder(nn.Module):
    def __init__(self,
                 src_vocab_size,
                 embed_size,
                 num_layers,
                 heads,
                 device,
                 forward_expansion,
                 dropout,
                 max_length,
                 ):
        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.num_layers = num_layers
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)
        
        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    heads,
                    dropout=dropout,
                    forward_expansion=forward_expansion,
                )
            for _ in range(num_layers)]
        )
        self.dropout = nn.Dropout(dropout)
    def forward(self, x, mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        out = self.dropout(self.word_embedding(x) + self.position_embedding(positions))

        for layer in self.layers:
            out = layer(out, out, out, mask)
        return out

class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion, device):
        super(DecoderBlock, self).__init__()
        self.attention = Attention(embed_size, heads)
        self.norm = nn.LayerNorm(embed_size)
        self.transformer_block = TransformerBlock(
            embed_size, heads, dropout, forward_expansion
        )
        self.dropout = nn.Dropout(dropout)
    def forward(self, x, value, key, src_mask, trg_mask):
        attention = self.attention(x,x,x,trg_mask)
        query = self.dropout(self.norm(attention + x))
        out = self.transformer_block(value, key, query, src_mask)
        return out
class Decoder(nn.Module):
    def __init__(self,
                 trg_vocab_size,
                 embed_size,
                 num_layers,
                 heads,
                 forward_expansion,
                 dropout,
                 max_length,
                 device):
        super(Decoder, self).__init__()
        self.embed_size = embed_size
        self.num_layers = num_layers
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)
        self.layers = nn.ModuleList(
            [
                DecoderBlock(embed_size, heads, dropout, forward_expansion, device)
                for _ in range(num_layers)
            ]
        )
        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x, enc_out, src_mask, trg_mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        out = self.dropout(self.word_embedding(x) + self.position_embedding(positions))

        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)
        out = self.fc_out(x)
        return out
class Transformer(nn.Module):
    def __init__(
            self,
            src_vocab_size,
            trg_vocab_size,
            src_pad_idx,
            trg_pad_idx,
            embed_size=256,
            forward_expansion=4,
            num_layers=6,
            heads=8,
            dropout=0,
            max_length=100,
            device="cpu",
    ):
        super(Transformer, self).__init__()
        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length,
        )

        self.decoder = Decoder(
            trg_vocab_size,
            embed_size,
            num_layers,
            heads,
            forward_expansion,
            dropout,
            max_length,
            device
        )
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device
    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        return src_mask.to(self.device)
    def make_trg_mask(self, trg):
        N, trg_len = trg.shape
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(
            N, 1, trg_len, trg_len
        )
        return trg_mask.to(self.device)
    def forward(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)

        enc_src = self.encoder(src, src_mask)
        out = self.decoder(trg, enc_src, src_mask, trg_mask)
        return out

In [12]:
x = torch.tensor([[1,2,3,4,5],[6,7,8,9,10]])
trg = torch.tensor([[1,2,3,4,5],[6,7,8,9,10]])

src_pad_idx = 0
trg_pad_idx = 0
src_vocab_size = 11
trg_vocab_size = 11

model = Transformer(src_vocab_size=src_vocab_size,
                    trg_vocab_size=trg_vocab_size,
                    src_pad_idx=src_pad_idx,
                    trg_pad_idx=trg_pad_idx)

out = model(x, trg[:,:-1])
print(out.shape)  # (N, trg_len, trg_vocab_size)


RuntimeError: The size of tensor a (5) must match the size of tensor b (8) at non-singleton dimension 3

In [17]:
class InputEmbeddings(nn.Module):
    def __init__(self, vocab_size, embed_size):
        super().__init__()
        self.embed_size = embed_size
        self.vocab_size = vocab_size
        self.embedding  = nn.Embedding(vocab_size, embed_size)
    
    def forward(self, x):
        return self.embedding(x) * math.sqrt(self.embed_size)

class PositionalEncoding(nn.Module):

    def __init__(self, embed_size:int, seq_len:int, dropout: float):
        super().__init__()
        self.embed_size = embed_size
        self.seq_len = seq_len
        self.dropout = nn.Dropout(dropout)

        # Matrix shape (embed_size, seq_len)
        pe = torch.zeros(embed_size, seq_len)
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1) # shape: (seq_len, 1)
        div_term = torch.exp(torch.arange(0, embed_size, 2).float() * -(math.log((1000.0))) / embed_size)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0) # Shape: (1, seq_len, embed_size)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :].requires_grad_(False)
        return self.dropout(x)
    
class LayerNorm(nn.Module):
    
    def __init__(self, eps: float = 1e-6):
        super().__init__()
        self.eps = eps
        self.alpha = nn.Parameter(torch.ones(1)) ## Multiplied
        self.bias = nn.Parameter(torch.zeros(1)) ## Added
    
    def forward(self, x):
        mean = x.mean(dim = -1, keepdims = True)
        std = x.std(dim = -1, keepdims = True)
        x = (x - mean) / (std + self.eps)
        return self.alpha * (x - mean) / (std + self.eps) + self.bias
    
class FeedForwardBlock(nn.Module):

    def __init__(self, embed_size: int, expansion: int, dropout: float) -> None:
        super().__init__()
        self.linear1 = nn.Linear(embed_size, embed_size * expansion)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(embed_size * expansion, embed_size)
    
    def forward (self, x):
        # (Batch, Seq_len, Embed_size) -> (Batch, Seq_len, embed_size * expansion) --> (Batch, Seq_len, Embed_size)
        x = self.linear1(x)
        x = nn.ReLU(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x

class AttentionBlock(nn.Module):

    def __init__(self, embed_size: int, heads: int, dropout: float) -> None:
        super().__init__()
        self.embed_size = embed_size
        self.heads = heads

        assert embed_size % heads == 0, "Embedding size must be divisible by heads"


        self.d_k = embed_size // heads
        self.w_q = nn.Linear(embed_size, embed_size) #W_q
        self.w_k = nn.Linear(embed_size, embed_size) #W_k
        self.w_v = nn.Linear(embed_size, embed_size) #W_v
        self.w_o = nn.Linear(embed_size, embed_size) #W_o

        self.dropout = nn.Dropout(dropout)
    
    @staticmethod
    def scaled_dot_product_attention(query, key, value, mask, dropout: nn.Dropout):
        # (Batch, Heads, sequence_length, embed_size)
        # scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(query.shape[-1])
        scores = torch.einsum("bhqd,bhkd->bhqk", [query, key]) / math.sqrt(query.shape[-1])
        if mask is not None: 
            scores = scores.masked_fill(mask == 0, float('-1e20'))
        if dropout is not None:
            scores = dropout(scores)

        # (batch, heads, sequence_len, d_k) --> (batch, heads, sequence_len, sequence_len)
        attention_score = torch.softmax(scores, dim=-1) # (batch, h, seq_len, seq_len)
        # out = torch.matmul(attention_score, value)
        torch.einsum("bhqk, bhkd -> bhqd", [attention_score, value])
        return out, attention_score
    
    def forward(self, q, k, v, mask):
        query = self.w_q(q)
        key = self.w_k(k)
        value = self.w_v(v)

        query = query.view(query.shape[0], query.shape[1], self.heads, self.d_k).transpose(1,2)
        key = key.view(key.shape[0], key.shape[1], self.heads, self.d_k).transpose(1,2)
        value = value.view(value.shape[0], value.shape[1], self.heads, self.d_k).transpose(1,2)

        x, self.attention_scores = AttentionBlock.scaled_dot_product_attention(query, key, value, mask, self.dropout)

        # (Batch, Heads, Seq_len, d_k) --> (Batch, Seq_len, Heads,  d_k) --> (Batch, Seq_len, Embed_size)
        x = x.tranpose(1,2).contiguous().view(x.shape[0], -1, self.heads * self.d_k)

        # (Batch, Seq_len, d_k) --> (Batch, Seq_len, Embed_size)
        return self.w_o(x)
    
class ResidualConnection(nn.Module):

    def __init__(self, dropout: float):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.norm = LayerNorm()
    
    def forward(self, x, sublayer):

        residual = x
        x = self.norm(x)
        x = sublayer(x)
        x = residual + self.dropout(x)

        # return x + self.dropout(sublayer(self.norm(x)))
        return x
    
class EncoderBlock(nn.Module):
    def __init__(self,
                 attention_block: AttentionBlock,
                 feed_forward_block: FeedForwardBlock,
                 dropout: float,
                 ):
        super().__init__()
        self.attention_block = attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.Modulelist([ResidualConnection(dropout) for _ in range(2)])
    
    def forward(self, x, src_mask):
        x = self.residual_connections[0](x, lambda x: self.attention_block(x, x, x, src_mask))
        x = self.residual_connections[1](x, self.feed_forward_block)
        return x
    
class Encoder(nn.Module):

    def __init__(self,
                 layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNorm()

    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

class DecoderBlock(nn.Module):

    def __init__(
        self,
        attention_block: AttentionBlock,
        cross_attention_block: AttentionBlock,
        feed_forward_block: FeedForwardBlock,
        dropout: float,
    ) -> None:
        super().__init__()
        self.attention_block = attention_block
        self.feed_forward_block = feed_forward_block
        self.cross_attention_block = cross_attention_block
        self.residual_connections = nn.Modulelist([ResidualConnection(dropout) for _ in range(3)]
    )
    
    def forward(self, x, enc_out, src_mask, target_mask):
        x = self.residual_connections[0](x, lambda x: self.attention_block(x,x,x,target_mask))
        x = self.residual_connections[1](x, lambda x: self.cross_attention_block(x,enc_out,enc_out,src_mask))
        x = self.residual_connections[2](x, self.feed_forward_block)
        return x
    
class Decoder(nn.Module):

    def __init__(self,
                 layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNorm()
    
    def forward(self, x, enc_out, src_mask, target_mask):
        for layer in self.layers:
            x = layer(x, enc_out, src_mask, target_mask)
        return self.norm(x)

class fullyconnected(nn.Module):
    def __init__(self, embed_size: int, target_vocab_size: int) -> None:
        super().__init__()
        self.fc = nn.Linear(embed_size, target_vocab_size)
    
    def forward(self, x):
        x = self.fc(x)
        x = torch.log_softmax(x, dim=-1)

        # return torch.log_softmax(self.fc(x), dim=-1)
        return x
class Transformer(nn.Module):

    def __init__(self,
                 encoder: Encoder,
                 decoder: Decoder,
                 input_embedding: InputEmbeddings,
                 target_embedding: InputEmbeddings,
                 positional_encoding: PositionalEncoding,
                 target_positional_encoding: PositionalEncoding,
                 fc: fullyconnected) ->  None:
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.input_embedding = input_embedding
        self.target_embedding = target_embedding
        self.positional_encoding = positional_encoding
        self.target_positional_encoding = target_positional_encoding
        self.fc = fc

    def encode(self, x, source_mask):
        x = self.input_embedding(x)
        x = self.positional_encoding(x)
        x = self.encoder(x, source_mask)
        return x
    
    def decode(self, x, enc_out, source_mask, target_mask):
        x = self.target_embedding(x)
        x = self.target_positional_encoding(x)
        x = self.decoder(x, enc_out, source_mask, target_mask)
        return x
    
    def forward(self, x):
        return self.fc(x)


def build_transformer(
        source_vocab_size: int,
        target_vocab_size: int,
        source_seq_len: int,
        target_seq_len: int,
        embed_size: int = 512,
        Nx: int = 6,
        heads: int = 8,
        expansion: int = 4
) -> None:
    source_embed = InputEmbeddings(embed_size, source_vocab_size)
    target_embed = InputEmbeddings(embed_size, target_vocab_size)

    source_positional_encoding = PositionalEncoding(embed_size, source_seq_len, dropout=0.1)
    target_positional_encoding = PositionalEncoding(embed_size, target_seq_len, dropout=0.2)

    encoder_blocks = [];
    for _ in range(Nx):
        encoder_attention_block = AttentionBlock(embed_size, heads, dropout=0.1)
        encoder_feed_forward_block = FeedForwardBlock(embed_size, expansion, dropout=0.2)
        encoder_block = EncoderBlock(encoder_attention_block, encoder_feed_forward_block, dropout=0.1)
        encoder_blocks.append(encoder_block)
    
    decoder_blocks = [];
    for _ in range(Nx):
        decoder_attention_block = AttentionBlock(embed_size, heads, dropout=0.1)
        decoder_cross_attention_block = AttentionBlock(embed_size, heads, dropout=0.1)
        decoder_feed_forward_block = FeedForwardBlock(embed_size, expansion, dropout=0.2)
        decoder_block = DecoderBlock(decoder_attention_block, decoder_cross_attention_block, decoder_feed_forward_block, dropout=0.1)
        decoder_blocks.append(decoder_block)
    
    encoder  = Encoder(nn.ModuleList(encoder_blocks))
    decoder = Decoder(nn.ModuleList(decoder_blocks))

    fc = fullyconnected(embed_size, target_vocab_size)
    transformer = Transformer(
        encoder,
        decoder,
        source_embed,
        target_embed,
        source_positional_encoding,
        target_positional_encoding,
        fc
    )

    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
        else:
            nn.init.zeros_(p)
    
    return transformer
